In [1]:
using CSV, JuMP, Gurobi, DataFrames;

In [2]:
GUROBI_ENV = Gurobi.Env();

Academic license - for non-commercial use only


# Reading Data

In [101]:
COMPETITION_KEY = "2019-12-01-41164"

"2019-12-01-41164"

In [102]:
PLAYERS_DATA_PATH = "../data/optimization_input_" * COMPETITION_KEY * ".csv";
OUTPUT_PATH = "../output/lineups_" * COMPETITION_KEY * ".csv";

In [103]:
players = CSV.read(PLAYERS_DATA_PATH);

In [104]:
players[:,:Game].pool

3-element Array{String,1}:
 "WAS@LAC"
 "UTA@TOR"
 "GS@ORL" 

# IP Inputs

In [105]:
# Fantasy points
fantasy_points = players[:,:prediction];
# Positions
PG = Int.(players[:,:Position].=="PG")
SG = Int.(players[:,:Position].=="SG")
SF = Int.(players[:,:Position].=="SF");
PF = Int.(players[:,:Position].=="PF")
C  = Int.(players[:,:Position].=="C")
# Salary
salary = players[:,:Salary];
# Injuries
injury = 1 .- ismissing.(players[:,Symbol("Injury Indicator")]) 
o_injury = Int.(Missings.coalesce.(players[:,Symbol("Injury Indicator")], 0).=="O")
q_injury = Int.(Missings.coalesce.(players[:,Symbol("Injury Indicator")], 0).=="Q")
p_injury = Int.(Missings.coalesce.(players[:,Symbol("Injury Indicator")], 0).=="P");

In [106]:
# Maximum players by position
MAX_PG = 2
MAX_SG = 2
MAX_SF = 2
MAX_PF = 2
MAX_C = 1
# Budget
BUDGET = 60000.0
# Number of lineups
NB_LINEUPS = 4
# Maximum of player overleaping
MAX_OVERLAP = 4;

# IP Formulation

In [107]:
# Number of players
nb_players = size(players)[1]

# Model
model = Model(solver=GurobiSolver(OutputFlag=0, GUROBI_ENV))

# Variable
@variable(model, z[i=1:nb_players], Bin)

# Constrains without the overleap constraint
@objective(model, Max, sum(fantasy_points.*z))
@constraint(model, sum(salary.*z) <= BUDGET)
@constraint(model, sum(PG.*z) == MAX_PG)
@constraint(model, sum(SG.*z) == MAX_SG)
@constraint(model, sum(SF.*z) == MAX_SF)
@constraint(model, sum(PF.*z) == MAX_PF)
@constraint(model, sum(C.*z) == MAX_C)
@constraint(model, z .<= (1 .- injury))

# Initialization (iteration=1)
solve(model)
x = Int.(getvalue(z))
lineups = players[x.==1,[:name, :Position, :team_key ,:prediction]]    
names!(lineups, Symbol.([string("Name_",1), string("Position_",1), string("Team_",1), string("FP_",1)]))  

# Rest of iterations
@constraint(model, sum(x.*z) <= MAX_OVERLAP)
for i=2:NB_LINEUPS
    solve(model)
    x = hcat(x,Int.(getvalue(z)))
    lineups_names = players[x[:,i].==1, [:name, :Position, :team_key, :prediction]]
    names!(lineups_names, Symbol.([string("Name_",i), string("Position_",i), string("Team_",i), string("FP_",i)]))    
    lineups = hcat(lineups, lineups_names)
    @constraint(model, sum(x[:,i].*z) <= MAX_OVERLAP)
end

CSV.write(OUTPUT_PATH, lineups);

In [108]:
DataFrame(lineups)

,Name_1,Position_1,Team_1,FP_1,Name_2,Position_2,Team_2
,String,String,String,Float64,String,String,String
1,Bradley Beal,SG,WAS,53.4317,Kawhi Leonard,PF,LAC
2,Paul George,SF,LAC,54.6104,Paul George,SF,LAC
3,Fred VanVleet,PG,TOR,43.2081,Donovan Mitchell,SG,UTA
4,Bojan Bogdanovic,SF,UTA,39.0917,Bojan Bogdanovic,SF,UTA
5,Evan Fournier,SG,ORL,39.2203,Mike Conley,PG,UTA
6,Aaron Gordon,PF,ORL,34.4912,Marc Gasol,C,TOR
7,D.J. Augustin,PG,ORL,28.7639,D.J. Augustin,PG,ORL
8,Mo Bamba,C,ORL,25.7915,Terrence Ross,SG,ORL
9,Omari Spellman,PF,GS,27.3356,Omari Spellman,PF,GS


In [109]:
lineups

,Name_1,Position_1,Team_1,FP_1,Name_2,Position_2,Team_2
,String,String,String,Float64,String,String,String
1,Bradley Beal,SG,WAS,53.4317,Kawhi Leonard,PF,LAC
2,Paul George,SF,LAC,54.6104,Paul George,SF,LAC
3,Fred VanVleet,PG,TOR,43.2081,Donovan Mitchell,SG,UTA
4,Bojan Bogdanovic,SF,UTA,39.0917,Bojan Bogdanovic,SF,UTA
5,Evan Fournier,SG,ORL,39.2203,Mike Conley,PG,UTA
6,Aaron Gordon,PF,ORL,34.4912,Marc Gasol,C,TOR
7,D.J. Augustin,PG,ORL,28.7639,D.J. Augustin,PG,ORL
8,Mo Bamba,C,ORL,25.7915,Terrence Ross,SG,ORL
9,Omari Spellman,PF,GS,27.3356,Omari Spellman,PF,GS


# Results

In [94]:
players[:,:Game].pool

3-element Array{String,1}:
 "WAS@LAC"
 "UTA@TOR"
 "GS@ORL" 

In [95]:
# Lineups
lineups

,Name_1,Position_1,Team_1,FP_1,Name_2,Position_2,Team_2
,String,String,String,Float64,String,String,String
1,Bradley Beal,SG,WAS,53.4317,Kawhi Leonard,PF,LAC
2,Paul George,SF,LAC,54.6104,Paul George,SF,LAC
3,Fred VanVleet,PG,TOR,43.2081,Donovan Mitchell,SG,UTA
4,Bojan Bogdanovic,SF,UTA,39.0917,Bojan Bogdanovic,SF,UTA
5,Evan Fournier,SG,ORL,39.2203,Mike Conley,PG,UTA
6,Aaron Gordon,PF,ORL,34.4912,Marc Gasol,C,TOR
7,D.J. Augustin,PG,ORL,28.7639,D.J. Augustin,PG,ORL
8,Mo Bamba,C,ORL,25.7915,Terrence Ross,SG,ORL
9,Omari Spellman,PF,GS,27.3356,Omari Spellman,PF,GS


In [96]:
# FP per lineup
aggregate(lineups[:,Symbol.([string("FP_",i) for i=1:NB_LINEUPS])], sum)

,FP_1_sum,FP_2_sum,FP_3_sum,FP_4_sum
,Float64,Float64,Float64,Float64
1,345.944,341.979,341.916,341.176
